In [1]:
from IPython.display import Markdown, display

import os

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('ggplot')

from pingouin import power_ttest, power_ttest2n, power_anova, power_chi2
from statsmodels.stats.gof import chisquare_effectsize

### Глава 6
#### Что значит «незначимо»: чувствительность критерия

**Чувствительность критерия** - способность критерия обнаружить различия. (Если критерий показывает отсутствие статистически значимых различий, это не доказывает факт их отсутствия!!!) Чувствительность зависит от величины различия, разброса данных и объема выборки.

##### ДВА РОДА ОШИБОК
В медицине:

* **Чувствительность** — это вероятность положительного результата у больного; она характеризует способность пробы выявлять болезнь.
* **Специфичность** — это вероятность отрицательного результата у здорового; можно сказать, что она характеризует способность пробы выявлять отсутствие болезни.

**Ошибка 1 рода** - найти различия там где их нет. (Ошибочно отклонить нулевую гипотезу). Уровень **значимости** критерия $\alpha$ - максимально приемлимая вероятность ошибки 1 рода.
**Ошибка 2 рода** - не найти различия там, где они есть. (Не отклонить нулевую гипотезу, когда она не верна). Вероятность ошибки второго рода обозначают $\beta$, т.о. чувствительность критерия - $1-\beta$.

_|Различия есть|Различий нет
-|-|-
Различия выявлены|Истинно положительный результат (чувствительность), $1-\beta$|Ложноположительный результат (ошибка 1 рода), $\alpha$
Различий не выявлено|Ложноотрицательный результат (ошибка 2 рода), $\beta$|Истинноотрицательный результат, $1-\alpha$

##### ЧЕМ ОПРЕДЕЛЯЕТСЯ ЧУВСТВИТЕЛЬНОСТЬ?
Факторы влияющие на чувствительность:
* Уровень значимости $\alpha$. Чем меньше $\alpha$, тем ниже чувствительность.
* Отношение величины различий к стандартному отклонению. Чем больше это отношение, тем чувствительнее критерий.
* Объем выборок. Чем больше объем, тем выше чувствительность критерия.

**Уровень значимости**:

Снижая $\alpha$, мы снижаем риск отвергнуть верную нулевую гипотезу, то есть найти различия (эффект) там, где их нет. Но тем самым мы снижаем и чувствительность — вероятность выявить имеющиеся на самом деле различия.

**Величина различий и стандартное отклонение**:

На чувствительность критерия влияет не абсолютная величина эффекта, а ее отношение к стандартному отклонению. это отношение $\phi=\delta / \sigma$ называется **параметром нецентральности**, где $\delta=\mu_1 - \mu_2$ - разность средних. Чем больше $\phi$, тем больше чувствительность.

**Объем выборки**:

С увеличением объема выборки чувствительность критерия увеличивается. Во первых с увеличением объема выборки растет число степеней свободы, а следовательно уменьшается критическое значение. Во вторых с увеличением объема выборки растет значение критерия.

##### КАК ОПРЕДЕЛИТЬ ЧУВСТВИТЕЛЬНОСТЬ КРИТЕРИЯ?

При заданной общей численности, равная численность групп обеспечивает максимальную чувствительность.

Если необходимо рассчитать чувствительность после проведения исследования, когда, не найдя статистически-значимых различий, необходимо определить, в какой степени это можно считать доказательством отсутствия эффекта, — тогда следует принять численность обеих групп равной меньшей из них. Такой расчет даст несколько заниженную оценку чувствительности, но убережет от излишнего оптимизма.

Все вышесказанное относится в основном к критерию Стьюдента, но может быть обобщено и на другие статистические критерии (определяется теми же самыми факторами, но ход вычислений различен).


**Критерий Стьюдента**:

Чувствительность представляет собой функцию от параметра нецентральности $\phi=\delta / \sigma$, уровня значимости $\alpha$ и объема выборки. 

В других источниках [Jacob Cohen, Statistical Power Analysis for the Behavioral Sciences (Second Edition), 1988] вместо $\phi$ используется обозначение $d$, обозначая ES(effect size) в стандартных единицах (предполагая равенство СО выборок).

*при решении задач используется пакет pingouin-stats, поэтому для избежания путаницы параметр будет обозначатья беквой d*

**Дисперсионный анализ**:

для определения чувствительности необходимо: число групп, их численность, уровень значимости и величина различий.

Вариант 1: Параметр нецентральности - $\phi=\cfrac{\delta}{\sigma}\sqrt{\cfrac{n}{2k}}$, где $\delta$ - минимальная величина различий между любыми двумя группами, $\sigma$ - стандартное отклонение в совокупности, $k$ — число групп, $n$ — численность каждой из них (предполагается равное число, для максимальной чувствительности при заданной общей численности) или численность минимальной группы.

Вариант 2: Параметр нецентральности - $\phi=\sqrt{\cfrac{\sum{(\mu_i-\mu)^2}}{k\sigma^2}}$, где $\mu_i$ - среднее в i-й группе, $\mu=\cfrac{\sum{\mu_i}}{k}$ - среднее по всем группам.

Так же нужно знать межгрупповое число степеней свободы - $\nu_{меж} = k-1$ и внутригрупповое число степеней свободы $\nu_{вну} = k(n – 1)$.

Первые два варианта требуют обращения к графическим диаграммам чувствительности зависящим от параметра нецентральности, $\alpha$, объема выборок и степеней свободы. Это не совсем удобно и современно.

Вариант 3: $d = \cfrac{m_{max} - m_{min}}{\sigma}$ далее возможны варианты того как средние групп расположены на отрезке от минимума к максимуму:

1. минимальная вариативность. две точки по краям и все остальные в центре. Тогда $f_1=d\sqrt{\cfrac{1}{2k}}$
2. промежуточный вариант. все точки равномерно расположена на отрезке. Тогда $f_2=\cfrac{d}{2}\sqrt{\cfrac{k+1}{3(k-1)}}$
3. максимальная вариативность. точки поровну расположены на краях отрезка. Тогда $f_3=d\cfrac{\sqrt{k^2-1}}{2k}$

Если распределение средних неизвестно, то лучше считать f=f_2. Из нее можно вычислить $\eta^2 = \frac{f^2}{1 + f^2}$, которую использовать как параметр ES(effect size) в пакете pingouin-stats.

**Таблицы сопряженности**:

 Параметр нецентральности задается формулой - $\phi=\sqrt{\cfrac{N}{(r-1)(c-1)+1}}w$, где $w = \sqrt{\sum_{i,j}{\cfrac{(p_{ij}-R_i C_j)^2}{R_i C_j}}} = \sqrt{\sum_{k=1}^{m=r*c}{\cfrac{(P_{1k}-P_{0k})^2}{P_{0k}}}}$ - effect size, $p_{ij}$ - доля в i-й строке j-го столбца, $P_{0k}$ - доля в k-й ячейке при выполнении нулевой гипотезы, $P_{1k}$ - доля в k-й ячейке при выполнении альтернативной гипотезы, m - число ячеек в таблице. Суммы по строкам обозначаются $R_i$, по столбцам — $С_j$. r — число строк, с — число столбцов и N — общее число наблюдений. Число степеней свободы $ν_{вну} = ∞$, $ν_{меж} = (r – 1)(с – 1)$

Для вычисления чувствительности при помощи пакета pingouin-stats, нужны только w, $ν_{меж}$, $\alpha$ и N.
 

In [2]:
def S(n1, n2, S1, S2):
    """
    Объединенная оценка стандартного отклонения
    """
    return (((n1 - 1)*S1**2 + (n2 - 1)*S2**2) / (n1 + n2 - 2))**.5

**6.1.**

Используя данные табл. 4.2, вычислите чувствительность критерия Стьюдента, способного обнаружить 50% различие наилучшего сердечного индекса между галотановой и морфиновой анестезией.

Метод|n|среднее|стандартное отклонение
-|-|-|-
Галотан|9|2.08|1.05
Морфин|16|1.75|0.88

In [3]:
delta = 2.08 * .5 # 50% от наилучшего сердечного индекса
sigma = S(9, 16, 1.05, 0.88)

d = delta/sigma

power = power_ttest(d=d, n=9)

print(f"delta = {delta},\nsigma = {sigma},\nd = {d}\npower = {power}")

delta = 1.04,
sigma = 0.9426143109089925,
d = 1.1033144606059455
power = 0.5943711507978157


Пакет pingouin позволяет рассчитать мощность для неравных по размеру выборок:

In [4]:
delta = 2.08 * .5 # 50% от наилучшего сердечного индекса
sigma = S(9, 16, 1.05, 0.88)

d = delta/sigma

power = power_ttest2n(nx=9, ny=16, d=d)

print(f"delta = {delta},\nsigma = {sigma},\nd = {d}\npower = {power}")

delta = 1.04,
sigma = 0.9426143109089925,
d = 1.1033144606059455
power = 0.7177514646874877


**Ответ**: Оценка чувствительности без учета различия в размере выборок — 59%, при учете различия - 72%

**6.2.**

По тем же данным определите, какова должна быть численность групп, чтобы с вероятностью 80% обнаружить 25% различие в наилучшем сердечном индексе.

In [5]:
delta = 2.08 * .25 # 25% от наилучшего сердечного индекса
sigma = S(9, 16, 1.05, 0.88)

d = delta/sigma

n = power_ttest(d=d, power=0.8)

print(n)

52.559809215688944


N = 53

**6.3.**

Используя данные табл. 4.2, определите чувствительность критерия Стьюдента для выявления изменения среднего артериального давления и общего периферического сосудистого сопротивления на 25%.

А) Среднее артериальное давление при наилучшем сердечном индексе, мм рт. ст.:
Метод|n|среднее|стандартное отклонение
-|-|-|-
Галотан|9|76.8|13.8
Морфин|16|91.4|19.6

Б) Общее периферическое сосудистое сопротивление при наилучшем сердечном индексе, дин с см-5
Метод|n|среднее|стандартное отклонение
-|-|-|-
Галотан|9|2210|1200
Морфин|2830|2830|1130

In [6]:
# А) Среднее артериальное давление:
delta = 76.8* .25
sigma = S(9, 16, 13.8, 19.6)

d = delta/sigma

power = power_ttest(d=d, n=9)
power2n = power_ttest2n(nx=9, ny=16, d=d)

print(f"delta = {delta},\nsigma = {sigma},\nd = {d}\npower = {power}\npower2n = {power2n} (с учетом разного размера групп)")

delta = 19.2,
sigma = 17.798290098624157,
d = 1.078755312651309
power = 0.5753481375396392
power2n = 0.6984023619781473 (с учетом разного размера групп)


In [7]:
# Б) Общее периферическое сосудистое сопротивление:
delta = 2210* .25
sigma = S(9, 16, 1200, 1130)

d = delta/sigma

power = power_ttest(d=d, n=9)
power2n = power_ttest2n(nx=9, ny=16, d=d)

print(f"delta = {delta},\nsigma = {sigma},\nd = {d}\npower = {power}\npower2n = {power2n} (с учетом разного размера групп)")

delta = 552.5,
sigma = 1154.829179914765,
d = 0.47842573569259705
power = 0.15921556247152183
power2n = 0.1961984981824466 (с учетом разного размера групп)


ОТВЕТ:

А) Среднее артериальное давление: чувствительность - 57% или при учете разного размера групп - 70%

Б) Общее периферическое сосудистое сопротивление: чувствительность - 16% или при учете разного размера групп - 20%

**6.4.**

В задаче 3.5 мы не обнаружили влияния внутривенного введения тетрагидроканнабинолов на антибактериальную защиту у крыс. Допустим, минимальное снижение, которое мы хотим выявить, составляет 20%, уровень значимости α = 0,05. Какова чувствительность критерия Стьюдента?

> **3.5.**
> Стремясь отделить действие тетрагидроканнабинолов от действия дыма, Г. Хубер и соавт. изучили их действие при внутривенном введении. После ингаляционного введения бактерий крысам  вводили  спиртовой  раствор  тетрагидроканнабинолов, контрольной группе вводили этиловый спирт. В обеих группах было по 36 животных. После введения тетрагидроканнабинолов доля погибших бактерий составила в среднем 51,4%, в контрольной группе — 59,4%. Стандартные ошибки среднего составили соответственно 3,2% и 3,9%. Позволяют ли эти данныеутверждать, что тетрагидроканнабинолы ослабляют антибактериальную защиту?

In [8]:
delta = 59.4* .2
sigma = S(36, 36, 3.2*6, 3.9*6)

d = delta/sigma

power = power_ttest(d=d, n=36, alternative="greater")

print(f"delta = {delta},\nsigma = {sigma},\nd = {d}\npower = {power}")

delta = 11.88,
sigma = 21.403270778084362,
d = 0.5550553522017949
power = 0.7540176336603862


Чувствительность ~ 75%

**6.5.**
По тем же данным определите, какой должна быть численность групп, чтобы обеспечить выявление снижения антибактериальной защиты на 20% с вероятностью 90% (уровень
значимости α = 0,05).

In [9]:
delta = 59.4* .2
sigma = S(36, 36, 3.2*6, 3.9*6)

d = delta/sigma

n = power_ttest(d=d, power=0.9, alternative="greater")

print(f"delta = {delta},\nsigma = {sigma},\nd = {d}\nn = {n}")

delta = 11.88,
sigma = 21.403270778084362,
d = 0.5550553522017949
n = 56.2833270439752


примерно по 56 крыс в каждой группе.

**6.6.**

Какой должна быть численность групп, чтобы с вероятностью 90% обнаруживать снижение летальности с 90 до 30%. Уровень значимости α = 0,05.

РЕШЕНИЕ (аналогии в главе 5):

Истинная доля - $p$, её выборочная оценка - $\hat{p}$, наименьшее различие долей, которое хотим выявить - $\Delta{p}$, объем каждой выборки - $n$.

Есль H0 верна, то величина $z=\cfrac{\Delta{\hat{p}}}{S_{\Delta{\hat{p}}}}$ подчиняется нормальному распределению. Кроме того, $\hat{p}_1$ и $\hat{p}_2$ - это две оценки одной и той же доли. Тогда ее объединенная оценка - $\hat{p} = (\hat{p}_1+\hat{p}_2)/2 = (0.3 + 0.9)/2 = 0.6$, а стандартная ошибка разности - $S_{\Delta{\hat{p}}}=\sqrt{\hat{p}(1-\hat{p})(\cfrac{1}{n} + \cfrac{1}{n})}=\cfrac{0.692}{\sqrt{n}}$.

Критическое значение $Z_{\alpha=0.05}=1.960$ (одностороннее). Т.о. $\Delta{\hat{p}}=z_{\alpha}S_{\Delta{\hat{p}}}=\cfrac{1.356}{\sqrt{n}}$

Истинные доли $p_1$ и $p_2$ составляют соответственно 0.3 и 0.9, тогда их разность $\Delta{р} = p_2 – p_1 = 0.6$, а ее стандартная ошибка - $S_{\Delta{p}}=\sqrt{\cfrac{p_1(1-p_1)}{n}+\cfrac{p_2(1-p_2)}{n}} = \cfrac{0.547}{\sqrt{n}}$.

Величиная $z=\cfrac{\Delta{\hat{p}}-\Delta{p}}{S_{\Delta{p}}}$ - подчиняется стандартному нормальному распределению, а необходимая чувствительность 90%. Величина $z$ правее которого лежит 90% всех значений. Это $z=–1.282$. 

Итак: $\Delta{\hat{p}}=\Delta{p}+z_\beta S_{\Delta{p}} = 0.6 + (-1.282)\cfrac{0.547}{\sqrt{n}} = 0.6 - \cfrac{0.701}{\sqrt{n}}$

Сравнивая обе оценки можно найти $n$

$\cfrac{1.356}{\sqrt{n}} = 0.6 - \cfrac{0.701}{\sqrt{n}}$

$n=11.7$

ОТВЕТ:в каждой группе должно быть по 12 больных.

**6.7.**
Используя данные из задачи 3.2, найдите вероятность обнаружить снижение максимальной объемной скорости середины выдоха на 0,25 л/с при уровне значимости α = 0,05.

> **3.2** 
> Курение считают основным фактором, предрасполагающим  к  хроническим  обструктивным  заболеваниям  легких.  Что касается пассивного курения, оно таким фактором обычно не считается. Дж. Уайт  и Г. Фреб усомнились в безвредности пассивного курения и исследовали проходимость дыхательных путей у некурящих,  пассивных  и  активных  курильщиков  (J.  White,  H. Froeb. Small-airways dysfunction in nonsmokers chronically exposed to tobacco smoke. N. Engl. J. Med., 302:720—723, 1980). Для характеристики состояния дыхательных путей взяли один из показателей функции внешнего дыхания — аксимальную объемную скорость середины выдоха которую измеряли во время профилактического  осмотра  сотрудников  Калифорнийского  университета в Сан-Диего. Уменьшение этого показателя — признак нарушения проходимости дыхательных путей. Данные обследования представлены в таблице. Размер каждой группы - 200 человек.
> **Максимальная объемная скорость средины выдоха, л/с**

Группа|Среднее|Стандартное отклонение
-|-|-
Некурящие работающие в помещении, где не курят | 3,17 | 0,74
Некурящие работающие в накуренном помещении | 2,72 | 0,71
Курящие выкуривающие небольшое число сигарет | 2,63 | 0,73
Курящие выкуривающие среднее число сигарет | 2,29 | 0,70
Курящие выкуривающие большое число сигарет | 2,12 | 0,72

> Можно  ли считать максимальную объемную скорость середины выдоха одинаковой во всех группах?

In [10]:
delta = 0.25
sigma = np.mean([0.74, 0.71, 0.73, 0.70, 0.72])

k = 5
f_1 = delta/sigma * (1/2/k)**.5  # минимальная вариативность
# f_2 = delta/sigma/2 * ((k+1)/3/(k-1))**.5

f = f_1
eta2 = (f**2/(1+f**2))

power_anova(eta_squared=eta2, k=k, n=200)

0.8023895278312116

Чувствительность - 80%

**6.8.**
Используя данные из задачи 3.3, найдите вероятность обнаружить увеличение уровня липопротеидов высокой плотности на 5 и 10 мг%. Уровень значимости α = 0,05.
> **3.3.**
> Низкий  уровень  холестерина  липопротеидов  высокой плотности  ( ХЛПВП)    —  фактор  риска  ишемической  болезни сердца. Некоторые исследования свидетельствуют, что физическая нагрузка может повысить уровень ХЛПВП. Дж. Хартунг и соавт. (G. Н. Hartung et al. Relation of diet to hidh-density liрoprotein cholesterol in middle-aged marathon runners, joggles, and inactive men. N. Engl. J. Med., 302:357—361, 1980) исследовали уровень ХЛПВП у бегунов-марафонцев, бегунов трусцой и лиц, не занимающихся спортом. Средний уровень ХЛПВП у лиц, не занимающихся спортом, составил 43,3 мг% (стандартное отклонение 14,2 мг%), у бегунов трусцой — 58,0 мг% (стандартное отклонение 17,7 мг%) и у марафонцев — 64,8 мг% (стандартное отклонение 14,3 мг%). Будем считать, что в каждой группе было по 70 человек. Оцените статистическую значимость различий между группами.

In [11]:
# А) на 5%

delta = 5
sigma = np.mean([14.2, 17.7, 14.3])

k = 3
f = delta/sigma * (1/2/k)**.5   # при k=3 f_1 == f_2
eta2 = (f**2/(1+f**2))

power_anova(eta_squared=eta2, k=k, n=70)

0.3817922217089257

In [12]:
# Б) на 10%

delta = 10
sigma = np.mean([14.2, 17.7, 14.3])

k = 3
f = delta/sigma * (1/2/k)**.5
eta2 = (f**2/(1+f**2))

power_anova(eta_squared=eta2, k=k, n=70)

0.937191588071193

На 5 мг% — 38%, на 10 мг% — 94%.

**6.9.**

По тем же данным определите, какой должна быть численность групп, чтобы изменение в 5 мг% можно было обнаружить с вероятностью 80% при уровне значимости α = 0,05.

In [13]:
delta = 5
sigma = np.mean([14.2, 17.7, 14.3])

k = 3
f = delta/sigma * (1/2/k)**.5   # при k=3 f_1 == f_2
eta2 = (f**2/(1+f**2))

power_anova(eta_squared=eta2, k=k, power=.8)

183.79901076029736

Ответ: каждая группа по 184 человека.

**6.10.**
В задаче 5.4 сравнивали частоту рецидивов инфекции мочевых путей после короткого курса того или иного антибактериального препарата. Допустим, минимальные различия, которые мы хотим выявить, таковы: в группах ампициллина и триметоприма/сулъфаметоксазола рецидив наступает у двух третей девочек, в группе цефалексина — у одной трети. Какой была бы чувствительность таблицы сопряженности при численности групп, указанной в задаче 5.4? Уровень значимости α = 0,05.
> **5.4.**<br>
> Р. Феннел и соавт. (R. Fennell et al. Urinary tract infections in children effect of short course antibiotic therapy on recurrence rate in children with previous infections. Clin. Pediatr., 19:121—124, 1980) сравнили эффективность трех антибиотиков при рецидивирующей инфекции мочевых путей у девочек 3—16 лет. После короткого курса одного из антибактериальных препаратов (назначенного случайным образом) в течение года делали повторные посевы мочи. При выявлении бактериурии констатировали рецидив. Были получены следующие результаты.

||Рецидив "+"|Рецидив "-"
-|-|-
Ампициллин|20|7
Триметоприм/сульфаметоксазол|24|21
Цефалексин|14|2

> Есть ли основания говорить о разной эффективности препаратов? Если да, то какой лучше?

In [14]:
n = np.array([27, 45, 16])  # число наблюдений по группам

x = n * np.array([[2/3, 2/3, 1/3], [1/3, 1/3, 2/3]])
P1 = x/x.sum()
P0 = P1.sum(0)*P1.sum(1).reshape(-1,1)  # expected frequencies, based on the marginal sums

w = chisquare_effectsize(P0.flatten(), P1.flatten())
power = power_chi2(dof=(P1.shape[0]-1)*(P1.shape[1]-1), w=w, n=n.sum())

print(f"power: {power*100:.3f}%")

power: 59.102%


Ответ: 59%

**6.11.**
Каким должен быть объем выборки, чтобы в задаче 6.10 чувствительность составила 80%?

In [15]:
power_chi2(dof=(P1.shape[0]-1)*(P1.shape[1]-1), w=w, power=.8)

139.16772809290313

Ответ: 139